<a href="https://colab.research.google.com/github/bigboom1253/e_commerce_recommendation_system_development/blob/master/Modeling/Content_Based_Filtering/Content%20Based%20Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
import datetime

path = "/content/drive/My Drive/최강1조-추천 시스템/데이터 수집/"
prod = pd.read_json(path + "all_prod_data.json")
# review = pd.read_json(path + "all_review_data.json")
# satisfaction = pd.read_json(path + "all_satisfaction_data.json")

# 컨텐츠 기반 추천 기획
1. 3개는 같은 카테고리, 나머지 3개는 상위 카테고리(현재 티몬에서 진행중) -> 개선 필요 -> 멘트 바꾸기
2. 같은 브랜드 추천 -> input: 1개의 아이템, output: 6개의 추천 아이템 (단, 같은 상품은 제외)
3. 사용자의 식품 관련 관심사 2-3개를 입력받아 추천
 - 다이어트, 나홀로족, 그리운 집밥, 피크닉, 브런치

타겟 소비자의 종류: 클릭, 장바구니, 구매

In [43]:
prod.drop_duplicates("prod_idx", keep="first", inplace = True) # 중복 제거
all_category = list(np.unique(prod.category)) # 원본 데이터의 모든 카테고리
category = [[] for i in range(10)] # 10개의 지정된 카테고리
other_category = [] # 분류 안 된 나머지
category_name = ["채소","정육·계란류","쌀·잡곡","수산물·건어물","스낵·견과·빵·떡","밀가루·오일·소스·장류","김치·젓갈·반찬류","과일","간편식·냉장·냉동","생수·음료·커피·유제품"]

for cate in all_category:
    if "채소" in cate: 
        category[0].append(cate)
    elif "정육·계란류" in cate or "축산물·축산가공" in cate: 
        category[1].append(cate)
    elif "쌀·잡곡" in cate or "쌀·잡곡·나물" in cate: 
        category[2].append(cate)
    elif "수산물·건어물" in cate or "해산물.건어물" in cate: 
        category[3].append(cate)
    elif "스낵·견과·빵·떡" in cate or "과자·간식" in cate or "견과 혼합세트" in cate: 
        category[4].append(cate)
    elif "밀가루·오일·소스·장류" in cate: 
        category[5].append(cate)
    elif "김치·젓갈·반찬류" in cate or "김치·반찬·국·쿠킹박스" in cate: 
        category[6].append(cate)
    elif "과일" in cate or "기타과일·세트" in cate or "곶감" in cate: 
        category[7].append(cate)
    elif "간편식·냉장·냉동" in cate or "즉석·간편식·면.소스" in cate or "라면·즉석밥·통조림" in cate: 
        category[8].append(cate)
    elif "생수·음료·커피·유제품" in cate:
        category[9].append(cate)
    else: 
        other_category.append(cate)

all_category_list = [] # 정상적인 카테고리 모음
for i in range(10):
    all_category_list.extend(category[i])

df = [] # 각 카테고리에 해당하는 데이터프레임 리스트
for i in range(10):
    df.append(prod[prod.category.apply(lambda x: x in category[i])])

for i in range(10):
    df[i]["category_num"] = i

prod = pd.concat([df[0], df[1], df[2], df[3], df[4], df[5], df[6], df[7], df[8]]) # 전처리 - 카테고리 번호 컬럼추가 및 다른 카테고리 제거

def select_keyword(lst, eda = False, category_ending_number=1): # 뒤에서 몇 번째 카테고리 뽑을건지
    if lst[-category_ending_number] != "":
        keyword = lst[-category_ending_number].replace("\n", ", ")
    else:
        keyword = lst[-category_ending_number-1].replace("\n", ", ")
    if eda == True:
        if len(keyword) > 20 : 
            return keyword[:10]
        else: 
            return keyword
    else:
        return keyword

def select_brand(lst):
    try:
        return re.compile("\[[가-힣]*\]").findall(lst)[0][1:-1]
    except:
        return ""

prod["small_category"] = prod.category.apply(lambda x: select_keyword(x, category_ending_number=1)) # 세부 카테고리 컬럼 생성
prod["big_category"] = prod.category.apply(lambda x: select_keyword(x, category_ending_number=2)) # 상위 카테고리 컬럼 생성
prod["brand"] = prod.title.apply(lambda x: select_brand(x)) # 브랜드 컬럼 생성

# 전처리 - 이상한 가격 제거
prod = prod[prod.price != 9999999]
prod = prod[prod.price != 0]

# 전처리 - 재고 없는거 날림
prod = prod[prod.stocks == 1]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### 관심사 컨텐츠 기반 추천
관심사와 카테고리를 매칭시킨 라벨링 데이터 활용

In [44]:
path_label = "/content/drive/My Drive/최강1조-추천 시스템/라벨링/라벨링_"

diet = pd.read_csv(path_label + "다이어트.csv").drop('Unnamed: 0', axis=1)
home = pd.read_csv(path_label + "그리운집밥.csv").drop('Unnamed: 0', axis=1)
# alone = pd.read_csv(path_label + "나홀로족.csv").drop('Unnamed: 0', axis=1)
# picnic = pd.read_csv(path_label + "피크닉.csv").drop('Unnamed: 0', axis=1)
# brunch = pd.read_csv(path_label + "브런치.csv").drop('Unnamed: 0', axis=1)

prod = pd.merge(prod, diet, on = 'small_category')
prod = pd.merge(prod, home, on = 'small_category')
# prod = pd.merge(prod, alone, on = 'small_category')
# prod = pd.merge(prod, picnic, on = 'small_category')
# prod = pd.merge(prod, brunch, on = 'small_category')

In [61]:
prod['interested'] = prod.다이어트.apply(lambda x: '다이어트 '*x) + \
                     prod.그리운집밥.apply(lambda x: '그리운집밥 '*x) + \
                     prod.나홀로족.apply(lambda x: '나홀로족 '*x) + \
                     prod.피크닉.apply(lambda x: '피크닉 '*x) + \
                     prod.브런치.apply(lambda x: '브런치'*x)

In [ ]:
def content_based_filtering(selected_prod_idx):
    
    selected_prod = prod[prod.prod_idx == selected_prod_idx]

    # 1. 같은 카테고리 상품 3개 + 상위 카테고리 상품 3개 buy_count 순으로 추천

    same_cate = list(prod[prod.prod_idx != selected_prod_idx][prod.small_category == selected_prod.small_category[0]].sort_values("buy_count", ascending=False).title[:3])
    upper_cate = list(prod[prod.small_category != selected_prod.small_category[0]][prod.big_category == selected_prod.big_category[0]].sort_values("buy_count", ascending=False).title[:3])
    display(pd.DataFrame({"클릭한 상품과 비슷해요": same_cate+upper_cate}))

    # 2. 같은 브랜드 상품 6개 buy_count 순으로 추천 + 부족하면 6개보다 적게 추천

    if selected_prod.brand[0] != "":
        brand_df = prod[prod.brand == selected_prod.brand[0]].sort_values("buy_count", ascending=False).title
        if len(brand_df) >= 6:
            display(pd.DataFrame({"판매자의 다른 제품이에요": list(brand_df[:6])}))
        else:
            display(pd.DataFrame({"판매자의 다른 제품이에요": list(brand_df)}))

In [ ]:
content_based_filtering(2865723582)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,클릭한 상품과 비슷해요
0,달콤해서 한입에 반한\n반시고구마 말랭이
1,[봉팔형님] 밤꿀고구마 중 1kg 외 꿀고구마 4종 크기별 모음 / 2개 구매시 3...
2,[컬러푸드] 봉팔형님 국내산 햇 밤꿀고구마 & 호박고구마 3kg 5kg 10kg
3,[무료배송] 2020년 수확 포근포근 수미 햇 감자 3kg - 20kg
4,"[트래블경북] 20년 포슬포슬 수미감자 10kg,20kg/대용량/식당용"
5,[무료배송] 2020년 국산 햇 감자 3kg


,판매자의 다른 제품이에요
0,[컬러푸드] 선착순20%할인쿠폰+해남 후기갑 꿀고구마 중상1kg 정품\n한입/특상/...
1,[컬러푸드] 에그파파 국내산 냉장 닭가슴살 1kg
2,[컬러푸드] 에그파파 국내산 냉장 100% 닭다리살 1kg
3,[컬러푸드] 20%쿠폰 도드람한돈 삼겹살 보쌈용 500g 외 모음전
4,[컬러푸드] 봉팔형님 국내산 햇 밤꿀고구마 & 호박고구마 3kg 5kg 10kg
5,[컬러푸드] 가야촌 한돈 냉장육 삼겹살
